## Exercício Redes Neurais Artificiais

Instruções:
Treinar uma Rede Neural Artificial do tipo Perceptron Multilayer para predizer os dados se um determinado paciente está doente ou não, com base em sintomas associados à doenças do coração.

Para tanto, a base de dados disponível no seguinte repositório deverá ser utilizada:  Heart_disease_statlog.csv.

O exercício deve ser entregue com o código fonte em Python utilizado para treinamento e teste da rede, bem como com as informações dos melhores hiperparêmetros utilizados e as métricas de desempenho obtidas. Os gráficos contendo as curvas de aprendizado também deverão ser disponibilizados.

    Na elaboração do exercício, atentar para:
    1. a necessidade de separar o conjunto de dados entre treinamento, validação e testes.
    2. normalizar os dados de entrada, sempre que julgar necessário.

### Importações e carregamento de dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

# Carregar o CSV
df = pd.read_csv('Heart_disease_statlog.csv')
print("Primeiras 5 linhas do dataset:")
print(df.head())

print("\nInformações gerais do dataset:")
print(df.info())

print("\nEstatísticas descritivas:")
print(df.describe())


Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'seaborn'

### Verifica nulos e distribui classes

In [ ]:
print("Verificação de valores nulos:")
print(df.isnull().sum())

print(f"\nShape do dataset: {df.shape}")
print(f"Número de features: {df.shape[1] - 1}")

# Visualizar distribuição das classes
plt.figure(figsize=(8, 6))
sns.countplot(x='target', data=df)
plt.title("Distribuição das classes (0 = saudável, 1 = doente)")
plt.xlabel("Classe")
plt.ylabel("Quantidade")
plt.show()

print(f"\nDistribuição das classes:")
print(df['target'].value_counts())


### Separação de dados

In [ ]:
# Features e alvo
X = df.drop('target', axis=1)
y = df['target']

print(f"Shape das features: {X.shape}")
print(f"Shape do target: {y.shape}")

# Normalização
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão treino/validação/teste: 60% treino, 20% validação, 20% teste
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)  # 0.25 * 0.8 = 0.2

print(f"\nDivisão dos dados:")
print(f"Treino: {X_train.shape[0]} amostras ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Validação: {X_val.shape[0]} amostras ({X_val.shape[0]/len(df)*100:.1f}%)")
print(f"Teste: {X_test.shape[0]} amostras ({X_test.shape[0]/len(df)*100:.1f}%)")


## Treinamento

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100,),  # pode testar (50,), (100,50), etc.
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)

print("Iniciando treinamento da rede neural...")
mlp.fit(X_train, y_train)
print(f"Treinamento concluído em {mlp.n_iter_} iterações")


### Curvas de aprendizado

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(mlp.loss_curve_, 'b-', linewidth=2)
plt.title("Curva de Perda durante o Treinamento", fontsize=14)
plt.xlabel("Épocas", fontsize=12)
plt.ylabel("Perda (Loss)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Perda final: {mlp.loss_curve_[-1]:.6f}")
print(f"Número de iterações: {mlp.n_iter_}")


### Avaliação

In [ ]:
# Predições
y_val_pred = mlp.predict(X_val)
y_test_pred = mlp.predict(X_test)

# Avaliação no conjunto de validação
print("=== RESULTADOS NO CONJUNTO DE VALIDAÇÃO ===")
print(f"Acurácia: {accuracy_score(y_val, y_val_pred):.4f}")
print("\nRelatório de classificação:")
print(classification_report(y_val, y_val_pred))

# Avaliação no conjunto de teste
print("\n=== RESULTADOS NO CONJUNTO DE TESTE ===")
print(f"Acurácia: {accuracy_score(y_test, y_test_pred):.4f}")
print("\nRelatório de classificação:")
print(classification_report(y_test, y_test_pred))

# Matriz de Confusão
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Validação
ConfusionMatrixDisplay.from_predictions(y_val, y_val_pred, ax=ax1, cmap='Blues')
ax1.set_title("Matriz de Confusão - Validação")

# Teste
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred, ax=ax2, cmap='Blues')
ax2.set_title("Matriz de Confusão - Teste")

plt.tight_layout()
plt.show()
